##Ejercicios ML con PySpark Vivo Clase 3


In [ ]:
!pip install pyspark -q

## Carga de datos y preparación del dataset
Importar el archivo "titanic.csv" a un DataFrame de PySpark y seleccionar solo las columnas "Pclass", "Age" y "Survived".

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import StringIndexer


spark = SparkSession.builder.master("local").appName("Titanic").getOrCreate()

url = "https://github.com/datasciencedojo/datasets/raw/master/titanic.csv"

spark.sparkContext.addFile(url)

titanic_data = spark.read.csv(SparkFiles.get("titanic.csv"), header=True)

# Agregar el sexo del pasajero a la evaluacion y ver si mejora la performance
# sex_column_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndexed').fit(titanic_data)
# titanic_data = sex_column_indexer.transform(titanic_data)

titanic_data_cleaned = titanic_data.na.drop().select(col("Pclass").cast(IntegerType()),
                                           col("Age").cast(IntegerType()),
                                           col("Survived").cast(IntegerType()))#,
                                          #col("SexIndexed").cast(IntegerType()))

## Creación de características (features) y etiquetas (labels)
Crear un vector de features usando VectorAssembler y renombrar la columna "Survived" como "label".

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["Pclass", "Age"]#, "SexIndexed"]
                            ,outputCol="features")
titanic_data_features = assembler.transform(titanic_data_cleaned)

titanic_data_features = titanic_data_features.withColumnRenamed("Survived", "label")

## Entrenamiento del modelo de clasificación y evaluación de su rendimiento
Entrenar un modelo de Regresión Logística usando el dataset de entrenamiento y evaluar su desempeño utilizando la métrica "Accuracy" en el conjunto de prueba.

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Dividir datos en conjuntos de entrenamiento y prueba
train_data, test_data = titanic_data_features.randomSplit([0.7, 0.3])

# Entrenamiento del modelo
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)

# Predicciones en el conjunto de test
predictions = lr_model.transform(test_data)

# Evaluación del rendimiento del modelo
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)

print("Test Accuracy: ", accuracy)

Test Accuracy:  0.7


##¿Qué pasa si agregamos una feature?
Agregar el sexo del pasajero a la evaluacion y ver si mejora la performance

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Inicializar SparkSession
spark = SparkSession.builder.master("local").appName("Titanic").getOrCreate()

# Cargar datos
url = "https://github.com/datasciencedojo/datasets/raw/master/titanic.csv"
spark.sparkContext.addFile(url)
titanic_data = spark.read.csv(SparkFiles.get("titanic.csv"), header=True)

# Transformación de la columna 'Sex'
sex_column_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndexed').fit(titanic_data)
titanic_data = sex_column_indexer.transform(titanic_data)

# Limpieza de datos y selección de características
titanic_data_cleaned = titanic_data.na.drop().select(
    col("Pclass").cast(IntegerType()),
    col("Age").cast(FloatType()),
    col("SexIndexed").cast(IntegerType()),
    col("Survived").cast(IntegerType())
)

# Creación de la columna de características
assembler = VectorAssembler(
    inputCols=["Pclass", "Age", "SexIndexed"],
    outputCol="features"
)
titanic_data_features = assembler.transform(titanic_data_cleaned)

# Renombrar columna para el modelo
titanic_data_features = titanic_data_features.withColumnRenamed("Survived", "label")

# Dividir datos en conjuntos de entrenamiento y prueba
train_data, test_data = titanic_data_features.randomSplit([0.7, 0.3])

# Entrenamiento del modelo
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)

# Predicciones en el conjunto de test
predictions = lr_model.transform(test_data)

# Evaluación del rendimiento del modelo
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)

print("Test Accuracy: ", accuracy)


Test Accuracy:  0.8243055555555555
